In [2]:
#%%time

#conda activate /projects/myenvs/candidates-env
import xarray as xr
import earthaccess
from earthaccess import Auth, Store, DataCollections, DataGranules
import datetime as dt
from datetime import timedelta
import time
import os
from pprint import pprint
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import folium
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import rioxarray
from scipy.spatial.distance import cdist
import matplotlib as mpl
from matplotlib.colors import ListedColormap

#set output and input file locations
output_dir = '/projects/my-public-bucket/outputs/quebec_lightning/' #CHANGE AS NEEDED FOR USER
file = f'/projects/my-public-bucket/scripts/quebec_lightning/Quebec_20_day_lightning_file_NO_CIFFC.csv'
lightn = pd.read_csv(file, header=0, usecols=["fireID", "t", "lt_lat", "lt_lon", "geometry"])
#gdf = gpd.read_file(file)

#search for only non-NULL values in lt_lat field
strikes = lightn.loc[~np.isnan(lightn.lt_lat), :]

#print(strikes)

# poly = lightn.loc[~pd.isna(lightn.geometry)]
# print(poly)

for i in strikes.index[:2]: 
    fire_id = strikes.loc[i, 'fireID']
    lat = strikes.loc[i, 'lt_lat']
    lon = strikes.loc[i, 'lt_lon']
    time = strikes.loc[i, 't'] 

#initialize variables
#make folders for each fire in the index if not already exists
    folderpath = f'{output_dir}/{fire_id}/' #CHANGE AS NEEDED FOR USER
    if not os.path.exists(folderpath):
        os.mkdir(folderpath)

    #print("The date and time is : ", time)

    date_object = dt.datetime.strptime(time, ("%Y-%m-%d %H:%M:%S.%f"))  
    #print(fire_id, time, lat, lon)

#for each index value, subtract 1 day and add 5 days to original time
    start_date = date_object + dt.timedelta(-1)
    end_date = date_object + dt.timedelta(+5)

    start_date = start_date.strftime("%Y-%m-%d")
    end_date = end_date.strftime("%Y-%m-%d")

    EXTENT = [lon - 0.5, lat - 0.5, lon + 0.5, lat + 0.5]
    START = start_date
    END = end_date

    print(fire_id, time, lon, lat, EXTENT, START, END) 

    # list1 = [fire_id]
    # list2 = [time]
    # df = pd.DataFrame(list1, list2)#, #columns=['Column1'])
    # print(df)
    # print(type(time))
    
    satellites = ['SNPP','NOAA20']
    
    #first check both satellites if there is a close match in time to the pyroCb
    #if there is a match for either, generate the outputs
    #if not, skip ahead to the next fire

    found_match = False
    
    for s in satellites:
        print(s)
        #s3_links, files = fetch_data(s)
        if s=='SNPP': products = ['VNP03IMG','VNP02IMG','VNP14IMG']
        elif s=='NOAA20': products = ['VJ103IMG','VJ102IMG','VJ114IMG']

        files = {}

        #query geolocation 03IMG
        results = earthaccess.search_data(
            short_name=products[0],
            bounding_box=(EXTENT[0],EXTENT[1],EXTENT[2],EXTENT[3]),
            temporal=(START, END),
            count=100
        )
        
        #use URLs to check if any are within 2 hours of pyroCb time (date_object)
        
        urls = [r.data_links()[0] for r in results]
        timestamps = []
        
        for url in urls:
            timestamp = url.split('.')[-5:-3]
            year = timestamp[0][1:5]
            day = timestamp[0][5:8]
            time = timestamp[1]

            timestamp_obj = dt.datetime.strptime(year+day+time[:2]+time[2:], '%Y%j%H%M')
            timestamps.append(timestamp_obj)
        
        #differences = [(date_object - t)/ dt.timedelta(hours=1) for t in timestamps]
        
        # if any(abs(d) < 2 for d in differences):
        #     found_match = True
        #     print(f'Found a close match for fire {fire_id}')
        #     break #skip ahead past NOAA20 / out of this satellite looop
         
    #if not found_match:
        #print(f'No match found for fire {fire_id}, skipping ahead')
        #continue #skip and continue on to the next fire
    
    #proceeds to this step unless found_match was not changed to True for either satellite
    print('proceeding to open data files...')
    
    for s in satellites:
        #s3_links, files = fetch_data(s)
        if s=='SNPP': products = ['VNP03IMG','VNP02IMG','VNP14IMG']
        elif s=='NOAA20': products = ['VJ103IMG','VJ102IMG','VJ114IMG']

        files = {}

        #query geolocation 03IMG
        results = earthaccess.search_data(
            short_name=products[0],
            bounding_box=(EXTENT[0],EXTENT[1],EXTENT[2],EXTENT[3]),
            temporal=(START, END),
            count=100
        )
        files[products[0]] = earthaccess.open(results)

        #science data 02IMG
        results = earthaccess.search_data(
            short_name=products[1],
            bounding_box=(EXTENT[0],EXTENT[1],EXTENT[2],EXTENT[3]),
            temporal=(START, END),
            count=100
        )
        files[products[1]] = earthaccess.open(results)

        
        #initiate cloud session - need to reauthenticate every hour :(
        auth = Auth() 
        #auth.login(strategy="interactive", persist=True) #RUN THIS THE FIRST TIME
        auth.login(strategy="netrc") #read credentials from previously saved ~/.netrc file

        store = Store(auth)
        fs = store.get_s3fs_session('LAADS') #daac or provider name
        
        product = products[2]

        Query = DataGranules().short_name(product).bounding_box(EXTENT[0],EXTENT[1],EXTENT[2],EXTENT[3]).temporal(START,END)

        print(Query.hits(), 'hits')
        cloud_granules = Query.get(800) #first 800 results
        print('cloud hosted', cloud_granules[0].cloud_hosted)

        s3_links = {}
        s3_links[product] = []
        for granule in cloud_granules:
            s3_links[product].extend(granule.data_links(access="in-region"))
        s3_links[product] = sorted(s3_links[product]) 
        files[product] = store.open(s3_links[product], provider="LAADS")

        print(product)
        pprint(files)
        

10286 2023-05-27 18:20:06.345 -74.666309 54.041834 [-75.166309, 53.541834, -74.166309, 54.541834] 2023-05-26 2023-06-01
SNPP
Granules found: 29
NOAA20
Granules found: 24
proceeding to open data files...
Granules found: 29
Opening 29 granules, approx size: 4.57 GB


QUEUEING TASKS | :   0%|          | 0/29 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/29 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/29 [00:00<?, ?it/s]

Granules found: 29
Opening 29 granules, approx size: 5.56 GB


...

21 hits
cloud hosted True


QUEUEING TASKS | :   0%|          | 0/21 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/21 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/21 [00:00<?, ?it/s]

VJ114IMG
{'VJ102IMG': [<File-like object HTTPFileSystem, https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/5201/VJ102IMG/2023/151/VJ102IMG.A2023151.0712.021.2023156153900.nc>,
              <File-like object HTTPFileSystem, https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/5201/VJ102IMG/2023/151/VJ102IMG.A2023151.1700.021.2023156155758.nc>,
              <File-like object HTTPFileSystem, https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/5201/VJ102IMG/2023/151/VJ102IMG.A2023151.1842.021.2023156155034.nc>,
              <File-like object HTTPFileSystem, https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/5201/VJ102IMG/2023/152/VJ102IMG.A2023152.0654.021.2023152171417.nc>,
              <File-like object HTTPFileSystem, https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/5201/VJ102IMG/2023/152/VJ102IMG.A2023152.0836.021.2023152170951.nc>,
              <File-like object HTTPFileSystem, https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/5201/VJ102IMG/2023/152/VJ102

In [ ]:
        all_dets = pd.DataFrame()

        for i in range(len(files[products[0]])): #VNP03IMG or VJ103IMG
            timestamp = files[products[0]][i].path.split('.')[-5:-3]
            print(timestamp)
            year = timestamp[0][1:5]
            day = timestamp[0][5:8]
            time = timestamp[1]
            date = dt.datetime.strptime(year+day, '%Y%j').strftime('%b %-d')
            acq_datetime_obj = dt.datetime.strptime(year+day+time[:2]+time[2:], '%Y%j%H%M')
            acq_datetime = acq_datetime_obj.strftime('%Y-%m-%d %H:%M:00 +00:00') 
            daytime = int(time) > 1500 #depends on timezone
            
            #calculate the time difference 
            diff = date_object - acq_datetime_obj
            diff = diff / dt.timedelta(hours=1)
            print(diff)
            
            try:
                #open 03IMG geolocation
                geo = xr.open_dataset(files[products[0]][i], engine='h5netcdf', group='geolocation_data')
                lon = geo['lt_lon'][:]
                lat = geo['lt_lat'][:]
                _, j = np.indices(geo.longitude.shape) #line and sample

                scene = (lon > EXTENT[0]) & (lon < EXTENT[2]) & (lat > EXTENT[1]) & (lat < EXTENT[3])

                #crop down the datasets for memory 
                indices = np.where(scene)
                x0 = indices[0].min()
                x1 = indices[0].max()
                y0 = indices[1].min()
                y1 = indices[1].max()

                lon = lon[x0:x1, y0:y1]
                lat = lat[x0:x1, y0:y1]
                j = j[x0:x1, y0:y1]

                #open 02IMG science data, i4 band
                data = xr.open_dataset(files[products[1]][i], engine='h5netcdf', group='observation_data')
                data = data.sel(number_of_lines=slice(x0,x1), number_of_pixels=slice(y0,y1))

                i4 = data['I04'] #xarray already encodes the scale factor and offset
                scale = data.I04.encoding['scale_factor']
                offset = data.I04.encoding['add_offset']
                i4 = (i4[:,:] - offset) / scale #return to raw values to use lookup table to temperature
                i4 = i4.astype(int)
                i4_bt = data['I04_brightness_temperature_lut'][:]
                i4_bt = i4_bt[i4]

                #get VNP14IMG
                match = [f for f in files[products[2]] if timestamp[0] and timestamp[1] in f.path][0]
                data = xr.open_dataset(match, phony_dims = 'sort')
                data = data.sel(phony_dim_1=slice(x0,x1), phony_dim_2=slice(y0,y1))
                daynight = data.DayNightFlag #string Day or Night

                qa = data.variables['algorithm QA'][:]
                fire = data.variables['fire mask'][:]  
                fires = (fire>6).values

            except:
                print('error with file',timestamp)
                #stop
                continue

            #look at QA flags data next over entire scene
            values, counts = np.unique(qa, return_counts=True)

            table = pd.DataFrame(index = values, columns=range(22,-1,-1)) #[22,21,...0]
            for i1 in table.index:
                b = np.binary_repr(i1, width=23)
                b = [int(s) for s in b]
                table.loc[i1, :] = b

            #report back all the pixels that have an 8 or 10 ~ background or candidate fires
            keep = table[(table.loc[:,8]==1) | (table.loc[:,10]==1)].index
            keep = (np.isin(qa[:], keep) | (fires))  #"fires" because some low conf are Test 16 pixel saturation


            #build pandas table for exporting, following VIIRS L2 columns
            i_dets = pd.DataFrame()
            i_dets['longitude'] = list(lon.values[keep])
            i_dets['latitude'] = list(lat.values[keep])
            i_dets['fire_mask'] = list(fire.values[keep])
            i_dets['daynight'] = daynight[0]
            i_dets['confidence'] = i_dets.fire_mask
            i_dets.confidence = i_dets.confidence.replace({0:'x', 1:'x', 2:'x', 3:'x', 4:'x', 5:'x', 6:'x', 7:'l', 8:'n', 9:'h'})
            i_dets['acq_date'] = date
            i_dets['acq_time'] = time
            i_dets['acq_datetime'] = acq_datetime
            i_dets['j'] = list(j[keep]) #sample number for pixel size lookup

            #crop down to defined extent
            i_dets = i_dets[(i_dets.longitude > EXTENT[0]) & (i_dets.longitude < EXTENT[2]) & (i_dets.latitude > EXTENT[1]) & (i_dets.latitude < EXTENT[3])]
            i_dets()